In [6]:
!pip install pandas numpy openai



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
pip install google-generativeai


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
df = pd.read_csv('booking_reviews copy.csv')
print(df.columns)


Index(['index', 'review_title', 'reviewed_at', 'reviewed_by', 'images',
       'crawled_at', 'url', 'hotel_name', 'hotel_url', 'avg_rating',
       'nationality', 'rating', 'review_text', 'raw_review_text', 'tags',
       'meta'],
      dtype='object')


In [17]:
import pandas as pd
import numpy as np


df = pd.read_csv('booking_reviews copy.csv')

def clean_dataset(df):
    cleaned_df = df[['reviewed_by', 'review_title', 'hotel_name', 'review_text', 'rating']]
    
    cleaned_df = cleaned_df.dropna()
    
    
    cleaned_df = cleaned_df.drop_duplicates()
    
    
    cleaned_df = cleaned_df.reset_index(drop=True)
    
    return cleaned_df


cleaned_df = clean_dataset(df)


cleaned_df.to_csv('cleaned_hotel_reviews.csv', index=False)

print("Dataset shape:", cleaned_df.shape)
print("\nSample of cleaned dataset:")
print(cleaned_df.head())

Dataset shape: (25983, 5)

Sample of cleaned dataset:
  reviewed_by                                       review_title  \
0      Kyrylo                                        Exceptional   
1     Dimitri  I highly recommend this b&b! We enjoyed it a lot!   
2    Virginia                                        Exceptional   
3      Kannan  My stay in the house was a experiencing bliss ...   
4         Sue  One bedroom apartment with wonderful view and ...   

               hotel_name                                        review_text  \
0         Villa Pura Vida  Everything was perfect! Quite, cozy place to r...   
1         Villa Pura Vida          Very friendly host and perfect breakfast!   
2  Hydro Palace Apartment  It was just what we wanted for a week by the b...   
3         Villa Pura Vida  My stay in the house was a experiencing bliss ...   
4  Hydro Palace Apartment  The building itself has a very musty smell in ...   

   rating  
0    10.0  
1     9.0  
2    10.0  
3    10.

In [15]:
pip install google-generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
!pip install python-dotenv



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install tenacity

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd
import numpy as np

def reduce_dataset(input_file, output_file, sample_size):
    print(f"Reading from {input_file}...")
    df = pd.read_csv(input_file)
    
    print(f"Total rows in original dataset: {len(df)}")
    
    if sample_size >= len(df):
        print("Sample size is larger than or equal to the dataset size. No reduction needed.")
        df.to_csv(output_file, index=False)
        print(f"Original dataset saved to {output_file}")
        return
    
    print(f"Reducing to {sample_size} rows...")
    
    # Randomly sample the data
    sampled_df = df.sample(n=sample_size, random_state=42)
    
    # Write the sampled data to a new CSV file
    sampled_df.to_csv(output_file, index=False)
    
    print(f"Reduced dataset saved to {output_file}")
    print(f"New dataset size: {len(sampled_df)} rows")

if __name__ == "__main__":
    input_file = 'cleaned_hotel_reviews.csv'
    output_file = 'reduced_hotel_reviews.csv'
    sample_size = 10 
    
    reduce_dataset(input_file, output_file, sample_size)

Reading from cleaned_hotel_reviews.csv...
Total rows in original dataset: 25983
Reducing to 10 rows...
Reduced dataset saved to reduced_hotel_reviews.csv
New dataset size: 10 rows


In [27]:
import os
from openai import OpenAI
import pandas as pd
from typing import Tuple
from dotenv import load_dotenv
import time
from tenacity import retry, stop_after_attempt, wait_exponential

# Load environment variables
load_dotenv()

# Initialize OpenRouter client
openai = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv('OPENROUTER_API_KEY')
)

@retry(
    wait=wait_exponential(multiplier=1, min=4, max=60),  # Wait between retries, increasing exponentially
    stop=stop_after_attempt(5)  # Stop after 5 attempts
)
def analyze_sentiment(text: str) -> Tuple[str, float]:
    """
    Analyze the sentiment of a given text using OpenRouter API with Gemini model.
    Returns a tuple of (sentiment_label, confidence_score)
    """
    try:
        # Add delay between requests to respect rate limits
        time.sleep(10)  
        
        completion = openai.chat.completions.create(
            model="google/gemini-2.0-flash-exp:free",
            messages=[
                {
                    "role": "system",
                    "content": "You are a sentiment analysis expert. Analyze the sentiment of the given text and respond with only two values: sentiment_label (positive, negative, or neutral) and confidence_score (between 0 and 1) separated by a comma."
                },
                {
                    "role": "user",
                    "content": text
                }
            ]
        )
        
        # Parse the response
        response = completion.choices[0].message.content
        sentiment_label, confidence_score = response.strip().split(',')
        confidence_score = float(confidence_score)
        
        return sentiment_label, confidence_score
    except Exception as e:
        print(f"Error analyzing sentiment for text: {e}")
        raise  

def process_in_batches(df: pd.DataFrame, batch_size: int = 10) -> pd.DataFrame:
    """
    Process the dataframe in batches and save progress periodically
    """
    df['sentiment_label'] = ''
    df['sentiment_score'] = 0.0
    
    total_reviews = len(df)
    
    for start_idx in range(0, total_reviews, batch_size):
        end_idx = min(start_idx + batch_size, total_reviews)
        batch = df.iloc[start_idx:end_idx]
        
        print(f"\nProcessing batch {start_idx//batch_size + 1} ({start_idx}-{end_idx} of {total_reviews})")
        
        for idx, row in batch.iterrows():
            try:
                label, score = analyze_sentiment(row['review_text'])
                df.at[idx, 'sentiment_label'] = label
                df.at[idx, 'sentiment_score'] = score
                print(f"Processed review {idx} - Sentiment: {label}, Score: {score:.2f}")
                
                # Save progress after each batch
                if (idx + 1) % batch_size == 0:
                    df.to_csv('sentiment_analysis_progress.csv', index=False)
                    print(f"Progress saved at review {idx + 1}")
                    
            except Exception as e:
                print(f"Failed to process review {idx}: {e}")
                # Save progress before exiting
                df.to_csv('sentiment_analysis_progress.csv', index=False)
                print("Progress saved due to error")
                continue
        
        time.sleep(500)  # 5 second delay between batches
    
    return df

if __name__ == "__main__":
    try:
        # Check if there's a progress file
        if os.path.exists('sentiment_analysis_progress.csv'):
            print("Found existing progress file. Resuming from last saved state...")
            df_with_sentiment = pd.read_csv('sentiment_analysis_progress.csv')
            # Filter out reviews that haven't been processed yet
            remaining_reviews = df_with_sentiment[df_with_sentiment['sentiment_label'] == '']
            if len(remaining_reviews) > 0:
                print(f"Continuing with {len(remaining_reviews)} remaining reviews...")
                df_with_sentiment = process_in_batches(remaining_reviews, batch_size=5)
            else:
                print("All reviews have been processed!")
        else:
            # Load the cleaned dataset
            print("Starting new analysis...")
            df_with_sentiment = pd.read_csv('reduced_hotel_reviews.csv')
            df_with_sentiment = process_in_batches(df_with_sentiment, batch_size=5)
        
        # Save final results
        output_file = 'hotel_reviews_with_sentiment_openrouter.csv'
        df_with_sentiment.to_csv(output_file, index=False)
        print(f"\nFinal results saved to {output_file}")
        
        # Display sample results
        print("\nSample results with sentiment analysis:")
        print(df_with_sentiment[['review_text', 'sentiment_label', 'sentiment_score']].head())
        
    except Exception as e:
        print(f"An error occurred: {e}")

Starting new analysis...

Processing batch 1 (0-5 of 10)
Error analyzing sentiment for text: Error code: 429 - {'error': {'message': 'Rate limit exceeded: experimental-models/google/gemini-2.0-flash-thinking-exp:free', 'code': 429}}
Error analyzing sentiment for text: Error code: 429 - {'error': {'message': 'Rate limit exceeded: experimental-models/google/gemini-2.0-flash-thinking-exp:free', 'code': 429}}
Error analyzing sentiment for text: Error code: 429 - {'error': {'message': 'Rate limit exceeded: experimental-models/google/gemini-2.0-flash-thinking-exp:free', 'code': 429}}
Error analyzing sentiment for text: Error code: 429 - {'error': {'message': 'Rate limit exceeded: experimental-models/google/gemini-2.0-flash-thinking-exp:free', 'code': 429}}
Error analyzing sentiment for text: Error code: 429 - {'error': {'message': 'Rate limit exceeded: experimental-models/google/gemini-2.0-flash-thinking-exp:free', 'code': 429}}
Failed to process review 0: RetryError[<Future at 0x1ef93f18fe0

KeyboardInterrupt: 